In [1]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code')

In [5]:
!pip install setuptools~=67.6.0
!pip install spacy~=3.5.0
!pip install numpy~=1.21.5
!pip install gensim~=4.1.2
!pip install networkx~=2.8.4
!pip install tomotopy
!pip install bertopic
!pip install igraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import network_creation
from gensim.models.phrases import Phraser, ENGLISH_CONNECTOR_WORDS
import preprocessing
import community_utils
import tomotopy as tp
import networkx as nx
import igraph as ig
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
from diversity_metrics import *
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel


In [130]:
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/text_datasets/europarl_en_train.txt", "r") as f:
    bbc_train = f.read().split("\n")
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/text_datasets/europarl_en_test.txt", "r") as f:
    bbc_test = f.read().split("\n")

In [77]:
import pickle
with open("/content/drive/My Drive/CMPUT 701 - Shraddha/Experiments - Code/ng1_master_object.obj", "rb") as f:
      master_object = pickle.load(f)

In [140]:
# reference - https://github.com/MaartenGr/BERTopic/issues/90
"""
MaartenGr commented on Apr 15, 2021
Good catch, I did not test for higher n-grams in the example. I made two changes:

Used the build_analyzer() instead of build_tokenizer() which allows for n-gram tokenization
Preprocessing is now based on a collection of documents per topic, since the CountVectorizer was trained on that data
Tested it with several ranges of n-grams and it seems to work now.
"""

topic_model = BERTopic(language="english", verbose=True, top_n_words=10)
topics, _ = topic_model.fit_transform(bbc_train)

Batches:   0%|          | 0/594 [00:00<?, ?it/s]

2023-03-18 04:56:55,841 - BERTopic - Transformed documents to Embeddings
2023-03-18 04:57:20,576 - BERTopic - Reduced dimensionality
2023-03-18 04:57:24,579 - BERTopic - Clustered reduced embeddings


In [141]:
print("Num of topic: ", len(set(topics)))

Num of topic:  255


In [142]:
topic_words_list = []

for i in range(len(set(topics))-2):
  single_topic = []
  topic = topic_model.get_topic(i)  # select the most frequent topic
  for t in topic:
    single_topic.append(t[0])
  topic_words_list.append(single_topic)

print(topic_words_list)

[['european', 'europe', 'union', 'eu', 'parliament', 'citizens', 'institutions', 'the', 'we', 'its'], ['commission', 'the', 'has', 'its', 'it', 'that', 'questions', 'this', 'to', 'is'], ['water', 'groundwater', 'surface', 'quality', 'directive', 'drinking', 'clean', 'waters', 'use', 'framework'], ['women', 'men', 'gender', 'representation', 'decisionmaking', 'equality', 'political', 'quotas', 'participation', 'female'], ['amendments', 'amendment', 'nos', 'no', 'tabled', 'accept', '22', 'proposed', 'committee', '10'], ['criminal', 'law', 'court', 'crime', 'justice', 'judicial', 'instance', 'legal', 'mutual', 'assistance'], ['funds', 'structural', 'budget', 'expenditure', 'fund', 'cohesion', 'additionality', 'budgetary', 'money', 'financial'], ['postal', 'service', 'services', 'post', 'liberalisation', 'mail', 'universal', 'office', 'offices', 'postman'], ['kosovo', 'albanian', 'serbs', 'albanians', 'serbia', 'ethnic', 'health', 'hospital', 'situation', 'reconstruction'], ['food', 'addit

In [143]:
import pandas as pd                     

# Preprocess Documents
documents = pd.DataFrame({"Document": bbc_train,
                          "ID": range(len(bbc_train)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

In [144]:
# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

In [145]:
# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

In [137]:
# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')

In [146]:
print("puw:", proportion_unique_words(topic_words, topk=10))
print("jd:", pairwise_jaccard_diversity(topic_words, topk=10))
print("irbo p=0.5:", irbo(topic_words, weight=0.5, topk=10))
print("irbo p=0.9:", irbo(topic_words, weight=0.9, topk=10))


for coherence in ["c_v", "c_npmi", "u_mass"]:
  for topn in [5, 10, 20]:
    cm = CoherenceModel(topics=topic_words,
                        texts=tokens,
                        dictionary=dictionary,
                        topn=topn,
                        coherence=coherence)
    score = cm.get_coherence()
    print(coherence, ":", score, "(topn=", topn, ")")


puw: 0.7440944881889764
jd: 0.9980555154251546
irbo p=0.5: 0.9984061061296496
irbo p=0.9: 0.9971176380820149
c_v : 0.7520233624207396 (topn= 5 )
c_v : 0.6336360277028632 (topn= 10 )
c_v : 0.6336360277028632 (topn= 20 )


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


c_npmi : nan (topn= 5 )
c_npmi : nan (topn= 10 )
c_npmi : nan (topn= 20 )
u_mass : nan (topn= 5 )
u_mass : nan (topn= 10 )
u_mass : nan (topn= 20 )


In [ ]:
#----------------------------------------
# Next section is just trial and error for hierarchical BerTopic

In [74]:
topic_model.get_topics()

{-1: [('the', 0.03300048477307588),
  ('of', 0.02670264198890454),
  ('in', 0.026272033684971066),
  ('to', 0.02620821872462214),
  ('and', 0.02248232675392839),
  ('that', 0.016871453402143088),
  ('it', 0.016870185241627516),
  ('is', 0.016832571349643277),
  ('said', 0.016282959480747133),
  ('for', 0.015615697734989059)],
 0: [('the', 0.03343347178674975),
  ('to', 0.028465644032686463),
  ('of', 0.024453819507135748),
  ('and', 0.022869408639656714),
  ('he', 0.022491697268050982),
  ('mr', 0.02244604343603879),
  ('said', 0.02069816485285114),
  ('in', 0.020228014228644536),
  ('be', 0.017570730008021097),
  ('that', 0.01729803166095689)],
 1: [('the', 0.029743273837662915),
  ('to', 0.02525952119937176),
  ('and', 0.024047013010114405),
  ('in', 0.021931403643613102),
  ('he', 0.021493431062159053),
  ('we', 0.02060833937620364),
  ('but', 0.02044680088626618),
  ('his', 0.01821226775926265),
  ('have', 0.01814591847399144),
  ('of', 0.017091406126290218)],
 2: [('film', 0.04834

In [ ]:
 # Evaluate metric(s)
topic_list = []
word_score_list = []

# Iterate over topics to create nested list of topics
for i in topic_model.get_topic_info()['Topic']:
  single_topic_list = []
  single_word_score_list = []
  for elem in topic_model.get_topic(i):
    single_topic_list.append(elem[0])
    single_word_score_list.append(elem[1])

  topic_list.append(single_topic_list)
  word_score_list.append(single_word_score_list)

In [ ]:
print(topic_list)
print(len(topic_list))
print(word_score_list)
print(len(word_score_list))

[['the', 'to', 'of', 'and', 'is', 'you', 'for', 'in', 'it', 'that'], ['team', 'game', 'he', 'season', 'games', 'play', 'players', 'hockey', 'year', 'the'], ['patients', 'of', 'msg', 'is', 'in', 'it', 'medical', 'and', 'health', 'my'], ['car', 'bike', 'cars', 'engine', 'miles', 'tires', 'for', 'it', 'the', 'my'], ['key', 'chip', 'clipper', 'encryption', 'keys', 'escrow', 'be', 'government', 'nsa', 'will'], ['gun', 'guns', 'firearms', 'of', 'militia', 'the', 'to', 'in', 'or', 'that'], ['card', 'monitor', 'video', 'vga', 'drivers', 'monitors', 'diamond', 'cards', 'windows', 'driver'], ['israel', 'israeli', 'arab', 'jews', 'arabs', 'palestinian', 'of', 'the', 'not', 'that'], ['db', 'windows', 'mov', 'dos', 'file', 'allocation', 'memory', 'paradox', 'nt', 'swap'], ['ram', 'ibm', 'os', 'motherboard', 'card', 'pc', 'monitor', 'software', 'drive', 'for'], ['fbi', 'koresh', 'batf', 'compound', 'fire', 'gas', 'they', 'was', 'were', 'the'], ['image', 'jpeg', 'gif', 'files', 'file', 'format', 'ima

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(bbc_train)


100%|██████████| 126/126 [00:01<00:00, 80.95it/s]


In [ ]:
hierarchical_topics

,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
125,252,the_to_of_and_in,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",249,of_the_that_to_and,251,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax_the_for_and_to,2.205003
124,251,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax_the_for_and_to,"[0, 2, 5, 7, 8, 10, 14, 16, 18, 19, 20, 21, 23...",250,windows_db_for_with_is,248,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax_the_in_for_to,1.533863
123,250,windows_db_for_with_is,"[5, 7, 8, 10, 18, 20, 23, 27, 30, 32, 37, 42, ...",245,db_drive_card_windows_with,239,window_image_jpeg_file_widget,1.362891
122,249,of_the_that_to_and,"[1, 3, 4, 6, 9, 11, 12, 13, 15, 17, 22, 24, 25...",230,god_that_is_of_not,247,the_of_to_and_that,1.355685
121,248,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax_the_in_for_to,"[0, 2, 14, 16, 19, 21, 29, 31, 34, 39, 41, 43,...",235,team_he_the_game_was,246,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax_for_is_to_the,1.315000
...,...,...,...,...,...,...,...,...
4,131,key_encryption_keys_chip_clipper,"[3, 54]",3,key_chip_clipper_encryption_keys,54,key_ripem_anonymous_des_privacy,0.406782
3,130,god_atheists_atheism_exist_atheist,"[28, 80]",80,why_exist_god_existence_genetic,28,god_atheists_atheism_atheist_argument,0.397467
2,129,space_launch_nasa_lunar_orbit,"[12, 13]",13,space_mission_orbit_nasa_probe,12,space_launch_moon_lunar_commercial,0.390430
1,128,jesus_god_he_hell_christ,"[15, 40]",15,jesus_god_hell_faith_heaven,40,jesus_he_tomb_christ_god,0.342835


In [ ]:
hierarchical_topics.Topics.values

array([list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126]),
       list([0, 2, 5, 7, 8, 10, 14, 16, 18, 19, 20, 21, 23, 27, 29, 30, 31, 32, 34, 37, 39, 41, 42, 43, 48, 49, 50, 51, 53, 57, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 79, 82, 83, 84, 85, 87, 88, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 118, 119, 121, 122, 123, 124, 126]),
       list([5, 7, 8, 10, 18, 20, 23, 27, 30, 32, 37, 42, 50, 51, 57, 65, 67, 69, 72, 74, 91, 92, 93,

In [ ]:
topic_model.get_topic_tree(hierarchical_topics)

'.\n├─of_the_that_to_and\n│    ├─god_that_is_of_not\n│    │    ├─church_pope_god_marriage_mary\n│    │    │    ├─church_pope_marriage_mary_god\n│    │    │    │    ├─■──oto_order_reuss_amorc_rosicrucian ── Topic: 76\n│    │    │    │    └─church_pope_marriage_mary_god\n│    │    │    │         ├─church_pope_marriage_schism_married\n│    │    │    │         │    ├─■──marriage_married_ceremony_church_commitment ── Topic: 61\n│    │    │    │         │    └─■──pope_church_schism_catholic_bishop ── Topic: 97\n│    │    │    │         └─mary_god_her_holy_father\n│    │    │    │              ├─■──mary_her_maria_she_sin ── Topic: 81\n│    │    │    │              └─■──god_holy_father_spirit_godhead ── Topic: 125\n│    │    │    └─easter_sabbath_resurrection_pagan_christians\n│    │    │         ├─■──sabbath_ceremonial_worship_christians_day ── Topic: 105\n│    │    │         └─■──easter_resurrection_pagan_goddess_celebration ── Topic: 104\n│    │    └─god_that_is_of_not\n│    │         ├─god

In [ ]:
pip install corextopic


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
